In [1]:
#!sudo pip3 install "foolbox==3.1.1"

In [2]:
import time
import torch
import torchvision
import foolbox
import eagerpy as ep
print(torch.__version__)
print(torchvision.__version__)
print(foolbox.__version__)
print(ep.__version__)

1.4.0+cu100
0.5.0+cu100
3.1.1
0.29.0


In [3]:
model = torchvision.models.resnet18(pretrained=True).eval()

In [4]:
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = foolbox.PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

In [5]:
# get data and test the model
# wrapping the tensors with ep.astensors is optional, but it allows
# us to work with EagerPy tensors in the following
images, labels = ep.astensors(*foolbox.samples(fmodel, dataset="imagenet", batchsize=16))
print(images.shape, labels.shape)

torch.Size([16, 3, 224, 224]) torch.Size([16])


In [6]:
print(foolbox.accuracy(fmodel, images, labels))

0.9375


In [7]:
%timeit foolbox.accuracy(fmodel, images, labels)

8.21 ms ± 54.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit foolbox.accuracy(fmodel, images[:1], labels[:1])

3.99 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
# apply the attack
start = time.time()
attack = foolbox.attacks.LinfPGD()
epsilons = [0.002]
advs, _, success = attack(fmodel, images, labels, epsilons=epsilons)
print(f"attack took {time.time() - start:.1f} seconds")

attack took 1.1 seconds


In [10]:
# calculate and report the robust accuracy
robust_accuracy = 1 - success.float32().mean(axis=-1)
for eps, acc in zip(epsilons, robust_accuracy):
    print(eps, acc.item())

0.002 0.0625


In [11]:
# apply the attack
start = time.time()
attack = foolbox.attacks.LinfPGD()
epsilons = [0.0, 0.001, 0.01, 0.03, 0.1, 0.3, 0.5, 1.0]
advs, _, success = attack(fmodel, images, labels, epsilons=epsilons)
print(f"attack took {time.time() - start:.1f} seconds")

attack took 9.0 seconds


In [12]:
# calculate and report the robust accuracy
robust_accuracy = 1 - success.float32().mean(axis=-1)
for eps, acc in zip(epsilons, robust_accuracy):
    print(eps, acc.item())

0.0 0.9375
0.001 0.375
0.01 0.0
0.03 0.0
0.1 0.0
0.3 0.0
0.5 0.0
1.0 0.0


In [13]:
## save images and labels for comparison with classic Foolbox
# import numpy as np
# np.save("images.npy", images.numpy())
# np.save("labels.npy", labels.numpy())